# Introduction

In this module, after finished data preparation (required `panel.pkl` file existence), we will fit the panel data model.

In [1]:
%load_ext autoreload
%autoreload


import pathlib
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)
    from linearmodels import PanelOLS, RandomEffects


%matplotlib inline
sns.set_style("darkgrid")

# Constants

In [2]:
ROOT_PATH = pathlib.Path("..")
DATA_PATH = ROOT_PATH.joinpath("_data")

SEED = 42

# Data preparation

In [3]:
panel = pd.read_pickle(DATA_PATH.joinpath("panel.pkl").resolve())
panel = panel.set_index(["Store", "Date"])

y = panel.pop("Sales")
X = panel

X.rename(columns={"Open": "Constant"}, inplace=True)

# Results cross-validation

For time series or panel data models cross-validation is sneaky.
We can't just split the data into k-folds and then average the results, because then we would assume that past data can be modeled with future data.
What we need to do is gradually add subsequent periods and validate on the first period following the most recent period taken into training.
We have a large data set, hence we will only validate on two random days of each month, starting from the half of the given data set.
It will give us enough comparison to achieve robust scores of estimator performance.

State-of-the-art approach would require from us to build two models, one with fixed and second with random effects and make a Hausman test to decide which effect include in the estimator but now we will pick the safer approach and take the always **consistent estimator** (but maybe not the most effective one) and use the **FE** model.

In [4]:
from datetime import timedelta

from sklearn.metrics import r2_score

from futsal.utils import set_local_seed


X_res = X.reset_index()
y_res = y.reset_index()

year_month_df = pd.DataFrame({
    "Year": X_res["Date"].dt.year,
    "Month": X_res["Date"].dt.month,
    "Day": X_res["Date"].dt.day
}).drop_duplicates().sort_values(by=["Year", "Month", "Day"])


thresh_date = year_month_df.iloc[int(year_month_df.shape[0] / 2)]
thresh_date = pd.Timestamp(
    year=thresh_date["Year"], month=thresh_date["Month"], day=1
)

superior_date = year_month_df.iloc[-1]
superior_date = pd.Timestamp(
    year=superior_date["Year"], month=superior_date["Month"], day=superior_date["Day"]
)

r2_scores = []
with set_local_seed(SEED):
    while thresh_date < superior_date:
        X_train = X_res[X_res["Date"] < thresh_date].set_index(["Store", "Date"])
        y_train = y_res[y_res["Date"] < thresh_date].set_index(["Store", "Date"])
        X_test = X_res[X_res["Date"] == thresh_date].set_index(["Store", "Date"])
        y_test = y_res[y_res["Date"] == thresh_date].set_index(["Store", "Date"])

        days_add = np.random.randint(15, 30)
        thresh_date += timedelta(days=days_add)
        
        fe_model = PanelOLS(
            y_train, 
            X_train
        )

        fe_res = fe_model.fit(cov_type="robust")
        
        y_pred = fe_model.predict(
            fe_res.params,
            exog=X_test[fe_res.params.index.tolist()]
        )
        r2_scores.append(r2_score(y_test, y_pred))

print(
    "Average cross-validation score is: {0:.3f} with {1:.3f} std."
    .format(
        np.mean(r2_scores),
        np.std(r2_scores)
    )
)

Average cross-validation score is: 0.434 with 0.381 std.


We can clearly see, as expected when predicting daily sales, which are characterized by bigger relative changes from one time period to another, that our predictions are burdened with a large standard deviation, while with a reasonably acceptable mean r2. That being sad we can finally fit our model on the full data set and interpret the coefficients assigned to the single variables.

In [5]:
fe_model = PanelOLS(
    y, 
    X
)
fe_res = fe_model.fit(cov_type="robust")
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  Sales   R-squared:                        0.5872
Estimator:                   PanelOLS   R-squared (Between):              0.7401
No. Observations:              843080   R-squared (Within):               0.3447
Date:                Thu, Aug 08 2019   R-squared (Overall):              0.5872
Time:                        05:59:52   Log-likelihood                -7.602e+06
Cov. Estimator:                Robust                                           
                                        F-statistic:                   3.747e+04
Entities:                        1115   P-value                           0.0000
Avg Obs:                       756.13   Distribution:               F(32,843047)
Min Obs:                       590.00                                           
Max Obs:                       940.00   F-statistic (robust):          2.444e+04
                            

Conclusions:

* `StateHoliday_a`: Ghe coefficient of the variable is not significantly different from zero - you might consider removing this variable.
* Days that most encourage customers to make large purchases are Mondays, Fridays and Sundays - you can suspect that the group is divided among those walking at the beginning of the week, when there is a lack of assortment on home shelves or at the very end on Friday, while it is common to go on Sunday for a larger group.
* People tend to shop more when the store is closed the next day or was closed the day before, hence the negative coefficients adjusted to the variables: `OpenTomorrow`, `OpenYesterday`.
* Temporary `Promo` seems to have a larger impact on the sales level unexpectedly unlike the `Promo2Active` feature, which suggests that sendind discount cards discourages people from shopping. More data would be needed to explain this phenomenon, because it is possible that these cards, sent in January, activate only in February and this variable should also be placed in a lagged approach.
* The number of customers the day before has a huge impact on sales. this may suggest some trends that should be considered over a longer period of time, e.g. by observing the average number of customers over the last 7 days or the dynamics of change today to the same day but week before.